In [160]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Use the inline backend for Jupyter notebooks
%matplotlib inline

# Setting up database connection
db_filename = 'Pladeo_survey_sensor.db'
connection = sqlite3.connect(db_filename)

# Fetch unique IDs from data
id_query = (f'''
SELECT DISTINCT RESPONSE.USER_ID
FROM (SELECT RESPONSE_A.USER_ID, RESPONSE_A.QUESTION_ID, RESPONSE_A.RESPONSE_1 FROM user_response_1 RESPONSE_A
LEFT JOIN (SELECT COUNT(*), USER_ID FROM user_response_1 GROUP BY USER_ID HAVING COUNT(*) = 3) RESPONSE_B
WHERE RESPONSE_A.USER_ID = RESPONSE_B.USER_ID) AS RESPONSE
LEFT JOIN (SELECT * FROM user_data) AS USER
ON RESPONSE.USER_ID = USER.USER_ID
''')  

ids_df = pd.read_sql(id_query, connection)
available_ids = ids_df['USER_ID'].tolist()

# Adding a default option to the dropdown options
dropdown_options = [('None', None)] + [(str(id), id) for id in available_ids]

# Creating the dropdown with the placeholder as the default selected value
id_selector = widgets.Dropdown(
    options=dropdown_options,
    description="Select an ID:",
    value=None  # Sets 'None' as the default, non-selectable option
)

# Create output widgets for displaying the graph and bar chart
output_graph = widgets.Output(layout={'border': '1px solid black', 'width': '400px', 'height': '300px'})
output_barchart = widgets.Output(layout={'border': '1px solid black', 'width': '300px', 'height': '200px'})

# Create an HBox to hold the output widgets
output_container = widgets.HBox(
    [output_graph, output_barchart],
    layout={'justify_content': 'space-between', 'margin': '20px 0 0 0'}  # Top margin for spacing
)

# Function to initialize the dashboard layout
def initialize_dashboard():
    # Clear outputs if no ID is selected
    with output_user_information:
        output_user_information.clear_output()
    with output_barchart:
        output_barchart.clear_output()
    with output_graph:
        output_graph.clear_output()

# Output widgets for graph and bar chart
output_user_information = widgets.Output()
output_barchart = widgets.Output()
output_graph = widgets.Output()

# Function for the bar chart based on the dropdown selection
def generate_user_information(selected_id):
        with output_user_information:
            output_user_information.clear_output()  # Clear previous outputs
			
            # Setting up database connection
            db_filename = 'Pladeo_survey_sensor.db'
            connection = sqlite3.connect(db_filename)
	
			# Query for values to response for the 3 Trials
            query = f'''
            SELECT DISTINCT USER.USER_ID, TRIM(UPPER(USER.FIRST_NAME)) AS FIRST_NAME, 
            CASE WHEN 
            USER.LAST_NAME IS NULL THEN ' '
            ELSE TRIM(UPPER(USER.LAST_NAME)) 
            END AS LAST_NAME, 
            CASE 
            WHEN USER.GENDER = 'F' THEN 'FEMALE'
            WHEN USER.GENDER = 'M' THEN 'MALE'
            ELSE NULL END AS GENDER, UPPER(USER.NATIONALITY) AS NATIONALITY
            FROM (SELECT * FROM user_data) AS USER
            LEFT JOIN (SELECT RESPONSE_A.USER_ID, RESPONSE_A.QUESTION_ID, RESPONSE_A.RESPONSE_1 FROM user_response_1 RESPONSE_A
            LEFT JOIN (SELECT COUNT(*), USER_ID FROM user_response_1 GROUP BY USER_ID HAVING COUNT(*) = 3) RESPONSE_B
            WHERE RESPONSE_A.USER_ID = RESPONSE_B.USER_ID) AS RESPONSE
            ON USER.USER_ID = RESPONSE.USER_ID
            WHERE USER.USER_ID = ?
    		'''
                    
            cursor = connection.cursor()
            cursor.execute(query, (selected_id,))
            results = cursor.fetchall()
            # Create label widgets for each user
    
            # label_widgets = []
            html_widgets = []

            # Creating HTML formatted strings
            for result in results:
                USER_ID, FIRST_NAME, LAST_NAME, GENDER, NATIONALITY = result
                LAST_NAME_UPPER = LAST_NAME.upper()
                user_id_html = f"<p style='margin: 15px; font-size: 24px; color: purple;'><strong>ID:</strong> {USER_ID}</p>"
                name_html = f"<p style='margin: 15px; font-size: 24px; color: purple;'><strong>FULL NAME:</strong> {FIRST_NAME} {LAST_NAME_UPPER}</p>"
                gender_html = f"<p style='margin: 15px; font-size: 24px; color: purple;'><strong>GENDER:</strong> {GENDER}</p>"
                nationality_html = f"<p style='margin: 15px; font-size: 24px; color: purple;'><strong>NATIONALITY:</strong> {NATIONALITY}</p>"

            # Append HTML widgets to the list
            html_widgets.extend([widgets.HTML(value=user_id_html),
                                 widgets.HTML(value=name_html),
                                 widgets.HTML(value=gender_html),
                                 widgets.HTML(value=nationality_html)])

            display(widgets.VBox(html_widgets))

# Function for the bar chart based on the dropdown selection
def generate_barchart(selected_id):
        with output_barchart:
            output_barchart.clear_output()  # Clear previous outputs
			
            # Setting up database connection
            db_filename = 'Pladeo_survey_sensor.db'
            connection = sqlite3.connect(db_filename)
            
			# Query the data for a specific ID
            id_to_plot = selected_id  
	
			# Query for values to response for the 3 Trials
            query = f'''
			SELECT RESPONSE.USER_ID, UPPER(TRIM(CONCAT(USER.FIRST_NAME, " ", USER.LAST_NAME))) AS USER_NAME, 
			CASE 
			WHEN RESPONSE.QUESTION_ID = 1 THEN 'T1' 
			WHEN RESPONSE.QUESTION_ID = 2 THEN 'T2' 
			WHEN RESPONSE.QUESTION_ID = 3 THEN 'T3' 
			ELSE 'E' 
			END as TRIAL_SESSION, RESPONSE.RESPONSE_1 AS USER_RESONSE
			FROM (SELECT RESPONSE_A.USER_ID, RESPONSE_A.QUESTION_ID, RESPONSE_A.RESPONSE_1 FROM user_response_1 RESPONSE_A
			LEFT JOIN (SELECT COUNT(*), USER_ID FROM user_response_1 GROUP BY USER_ID HAVING COUNT(*) = 3) RESPONSE_B
			WHERE RESPONSE_A.USER_ID = RESPONSE_B.USER_ID) AS RESPONSE
			LEFT JOIN (SELECT * FROM user_data) AS USER
			ON RESPONSE.USER_ID = USER.USER_ID
			WHERE RESPONSE.USER_ID = '{id_to_plot}'
    		'''

            
			# Step 3: Load the data into a DataFrame
            df = pd.read_sql_query(query, connection)
			
			# Extract unique name (assuming all rows have the same names)
            unique_name1 = df['USER_NAME'].unique()[0]  # Get the first (and only) unique name
            unique_name = unique_name1.upper()
			
			# Define custom y-axis co-ordinates
            custom_y_values = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]  # Customize as needed
			
			# Define colors based on value thresholds
            colors = ['orchid' if val < 0 else 'skyblue' if val > 0 else 'red' for val in df['USER_RESONSE']]
			
			# Step 4: Create bar chart
            plt.figure(figsize=(5, 5))
            bars = plt.bar(df['TRIAL_SESSION'], df['USER_RESONSE'], width=0.4, color=colors)  
			
            plt.yticks(custom_y_values)  # Set the y-axis coordinates to your custom values
			
			# Set custom y-axis limits based on the defined coordinates
            plt.ylim(min(custom_y_values), max(custom_y_values))
			
            plt.xlabel('Trials')
            plt.ylabel('User Response')
            plt.title(unique_name)
            plt.grid(axis='y', color='lightgrey', linestyle='--')
			
			# Label only the bars with zero values
            for i, bar in enumerate(bars):
            	height = bar.get_height()
            	if height == 0:  # Only label bars with a value of zero
            		plt.text(bar.get_x() + bar.get_width() / 2, height - 0.1, '0', 
    						ha='center', va='bottom', fontsize=10, color='grey')
			
            plt.show()


# Function for the graph based on the dropdown selection
def generate_graph(selected_id): 
    with output_graph:
        output_graph.clear_output()  # Clear previous outputs
        
        # Database filename
        db_filename = 'Pladeo_survey_sensor.db'
        
        # Query the data for a specific ID
        id_to_plot = selected_id  # Except 3, 5, 25, 26 and 33
        
        # List of queries for different graphs
        queries = [
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS813 as TGS813
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS813 as TGS813
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            ),
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS822 as TGS822
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS822 as TGS822
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            ),
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS832 as TGS832
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS832 as TGS832
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            ),
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2600 as TGS2600
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2600 as TGS2600
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            ),
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2603 as TGS2603
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2603 as TGS2603
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            ),
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2610 as TGS2610
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2610 as TGS2610
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            ),
            (
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2620 as TGS2620
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-R","1-R","2-R","3-R") AND PEAK.USER_ID = '{id_to_plot}'
                ''',
                f'''
                SELECT PEAK.USER_ID as USER_ID, LABEL.LABEL as RLABEL, 
                CASE 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = 'C' THEN 'Control' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '1' THEN 'T1' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '2' THEN 'T2' 
                WHEN SUBSTR(LABEL.LABEL, 1, 1) = '3' THEN 'T3' 
                ELSE 'E' 
                END as LABEL, PEAK.TGS2620 as TGS2620
                FROM (SELECT* FROM peak_details) AS PEAK
                LEFT JOIN (SELECT * FROM sample_label) AS LABEL
                ON PEAK.LABEL_ID = LABEL.LABEL_ID
                WHERE LABEL.LABEL IN ("C-L","1-L","2-L","3-L") AND PEAK.USER_ID = '{id_to_plot}'
                '''
            )
        ]
	    
        # List of y-axis column names corresponding to each query
        y_axis_columns = ['TGS813', 'TGS822', 'TGS832', 'TGS2600', 'TGS2603', 'TGS2610', 'TGS2620']  
	    
        # Connect to the database
        connection = sqlite3.connect(db_filename)
        
        # Initialize lists to collect all y-axis values
        # all_y_values = []
        all_y_values = [0.9, 0.94, 0.98, 1.02, 1.06, 1.10, 1.14, 1.18, 1.22, 1.26, 1.300]
        # Calculate equidistant positions for the labels (e.g., 0, 1, 2 for three labels)
        positions = np.linspace(0.9, 1.300, len(all_y_values))
        
        # First pass to collect all y values
        for c, (query1, query2) in enumerate(queries):
            df1 = pd.read_sql(query1, connection)
            df2 = pd.read_sql(query2, connection)
            all_y_values.extend(df1[y_axis_columns[c]].values)  # Collecting values from y-axis column: Query 1
            all_y_values.extend(df2[y_axis_columns[c]].values)  # Collecting values from y-axis column: Query 2
        
        # Determine y-axis limits
        # y_min = min(all_y_values)  
        # y_max = max(all_y_values)
        
        # Reset connection for plotting
        connection.close()
        connection = sqlite3.connect(db_filename)
        
        # Loop to generate each graph
        for i, (query1, query2) in enumerate(queries):
            # Fetch data for both queries
            df1 = pd.read_sql(query1, connection)
            df2 = pd.read_sql(query2, connection)
               
            # Create a figure for the current graph
            plt.figure(figsize=(10, 4), dpi=100)
            plt.plot(df1['LABEL'], df1[y_axis_columns[i]], marker='.', markersize=5, label='Right', color='red')
            plt.plot(df2['LABEL'], df2[y_axis_columns[i]], marker='.', markersize=5, label='Left', color='green')
        
            # Set y-axis limits
            plt.ylim(min(all_y_values), max(all_y_values))  # Set the same y-axis limits for all graphs
            
            # Setting the font size for x and y tick labels
            plt.xticks(fontsize=8)  # Adjust font size of x-axis labels
            plt.yticks(positions, fontsize=8)  # Adjust font size of y-axis labels
            plt.margins(x=0.05, y=0.1)  # Adds padding around the x and y axis
                
            # Add titles and labels
            plt.xlabel('Sample', fontsize=12)
            plt.ylabel('Peak Value', fontsize=12)
            plt.title(f'{y_axis_columns[i]} Sensor Graph', fontsize=16)
            plt.legend()
            plt.grid(False)
            plt.show()
	    

# Define function to call both graph and bar chart functions on ID selection
def on_id_change(change):
    selected_id = change['new']
    if selected_id is None:
        # If no ID is selected, clear both outputs and re-initialize the dashboard
        initialize_dashboard()
        return

    # Generate both the graph and the bar chart
    generate_user_information(selected_id)
    generate_barchart(selected_id)
    generate_graph(selected_id)

# Link the dropdown to the update function
# widgets.interactive(generate_graph, selected_id=id_selector)

# Link the dropdown to the update function
# widgets.interactive(generate_barchart, selected_id=id_selector)

# Link the function to the dropdown widget
id_selector.observe(on_id_change, names='value')

# Display the widgets with VBox to control layout
display(widgets.VBox([id_selector, widgets.HBox([output_user_information, output_barchart, output_graph])]))

# Initialize the dashboard layout
initialize_dashboard()

connection.close()